In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
genome_score=pd.read_csv(path_1)
genome_tags=pd.read_csv(path_2)


#Memory Based Collaborative Filtering - Garvika 202318061




#ITEM ITEM COLLABORATIVE


In [ ]:
movie_path='/content/drive/MyDrive/RecSys/movie.csv'
rating_path='/content/drive/MyDrive/RecSys/rating.csv'

In [ ]:
movie=pd.DataFrame(pd.read_csv(movie_path,usecols=[0,1]))
rating=pd.DataFrame(pd.read_csv(rating_path,usecols=[0,1,2]))

In [ ]:
rating.columns

Index(['userId', 'movieId', 'rating'], dtype='object')

In [ ]:
movie.columns

Index(['movieId', 'title'], dtype='object')

In [ ]:
merged_data=rating.merge(movie,on='movieId')

In [ ]:
merged_data

,userId,movieId,rating,title
0,1,2.0,3.5,Jumanji (1995)
1,5,2.0,3.0,Jumanji (1995)
2,13,2.0,3.0,Jumanji (1995)
3,29,2.0,3.0,Jumanji (1995)
4,34,2.0,3.0,Jumanji (1995)
...,...,...,...,...
225277,1555,406.0,3.0,Federal Hill (1994)
225278,1557,138.0,3.0,"Neon Bible, The (1995)"
225279,1559,84312.0,3.0,Home Alone 4 (2002)
225280,1559,90620.0,5.0,Mulan (2009)


In [ ]:
merged_data=merged_data.dropna(axis=0,subset=['title'])
rating_count = merged_data.groupby('title')['rating'].count().reset_index().rename(columns={'rating': 'totalratings'})[['title', 'totalratings']]
merged_data=merged_data.merge(rating_count, on='title')

In [ ]:
rating_count

,title,totalratings
0,'Hellboy': The Seeds of Creation (2004),2
1,'Neath the Arizona Skies (1934),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),2
4,'Til There Was You (1997),8
...,...,...
10307,eXistenZ (1999),60
10308,xXx (2002),50
10309,xXx: State of the Union (2005),10
10310,¡Three Amigos! (1986),68


In [ ]:
merged_data

,userId,movieId,rating,title,totalratings
0,1,2.0,3.5,Jumanji (1995),248
1,5,2.0,3.0,Jumanji (1995),248
2,13,2.0,3.0,Jumanji (1995),248
3,29,2.0,3.0,Jumanji (1995),248
4,34,2.0,3.0,Jumanji (1995),248
...,...,...,...,...,...
225277,1555,406.0,3.0,Federal Hill (1994),1
225278,1557,138.0,3.0,"Neon Bible, The (1995)",1
225279,1559,84312.0,3.0,Home Alone 4 (2002),1
225280,1559,90620.0,5.0,Mulan (2009),1


##Creating a popularity threshold to filter out non popular itmes for collaborative filtering

In [ ]:
merged_data.describe()

,userId,movieId,rating,totalratings
count,225282.000000,225282.000000,225282.000000,225282.000000
mean,776.091876,8414.530038,3.499931,151.195311
std,434.447831,18951.003232,1.072091,157.842043
min,1.000000,1.000000,0.500000,1.000000
25%,410.000000,866.000000,3.000000,34.000000
50%,768.000000,2125.000000,3.500000,95.000000
75%,1147.000000,4447.000000,4.000000,218.000000
max,1560.000000,130219.000000,5.000000,773.000000


##we can see that on an average each product has atleast 151 ratings. So it would be safe to remove the items with ratings less than 150 labelling them as not so popular items.

In [ ]:
popular_threshold=150
popular_items = merged_data.query('totalratings >= @popular_threshold')

In [ ]:
popular_items

,userId,movieId,rating,title,totalratings
0,1,2.0,3.5,Jumanji (1995),248
1,5,2.0,3.0,Jumanji (1995),248
2,13,2.0,3.0,Jumanji (1995),248
3,29,2.0,3.0,Jumanji (1995),248
4,34,2.0,3.0,Jumanji (1995),248
...,...,...,...,...,...
181651,1523,5989.0,4.0,Catch Me If You Can (2002),193
181652,1525,5989.0,3.5,Catch Me If You Can (2002),193
181653,1528,5989.0,4.5,Catch Me If You Can (2002),193
181654,1530,5989.0,5.0,Catch Me If You Can (2002),193


##KNN Implementation on User Itme Interaction matrix

In [ ]:
popular_items=popular_items.drop_duplicates(['userId','movieId','rating','title','totalratings'])

##Making pivot(interaction) table

In [ ]:
popular_items_pivot=popular_items.pivot(index='title',columns='userId',values='rating').fillna(0)

In [ ]:
popular_items_pivot

userId,1,2,3,4,5,6,7,8,9,10,...,1551,1552,1553,1554,1555,1556,1557,1558,1559,1560
title,,,,,,,,,,,,,,,,,,,,,
12 Angry Men (1957),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),3.5,5.0,5.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.5,3.0,0.0,0.0,0.0,0.0,0.0
"Abyss, The (1989)",0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ace Ventura: Pet Detective (1994),0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
Ace Ventura: When Nature Calls (1995),0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Willy Wonka & the Chocolate Factory (1971),0.0,0.0,5.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,...,0.0,3.5,0.0,0.0,3.0,4.0,4.0,0.0,0.0,0.0
"Wizard of Oz, The (1939)",3.5,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X-Men (2000),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [ ]:
user_item_matrix=csr_matrix(popular_items_pivot.values)
user_item_matrix

<316x1542 sparse matrix of type '<class 'numpy.float64'>'
	with 82814 stored elements in Compressed Sparse Row format>

In [ ]:
model_knn=NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(user_item_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
pick_random_title=np.random.choice(popular_items_pivot.shape[0])
pick_random_title


63

In [ ]:
similarity, index = model_knn.kneighbors(popular_items_pivot.iloc[pick_random_title, :].values.reshape(1, -1), n_neighbors=15)
popular_items_pivot.index[pick_random_title]

"City Slickers II: The Legend of Curly's Gold (1994)"

##Top 15 Reccomended items based on item item similarity

In [ ]:
for i in range(0,len(similarity.flatten())):
  if i ==0:
    print('Reccomendation for {0}:\n'.format(popular_items_pivot.index[pick_random_title]))
  else:
    print('{0}: {1}, with similarity {2}'.format(i, popular_items_pivot.index[index.flatten()[i]], similarity.flatten()[i]))


Reccomendation for City Slickers II: The Legend of Curly's Gold (1994):

1: Beverly Hills Cop III (1994), with similarity 0.5301795195034507
2: Coneheads (1993), with similarity 0.5470080766851907
3: Santa Clause, The (1994), with similarity 0.5524399383354717
4: Cliffhanger (1993), with similarity 0.5629829045641465
5: Net, The (1995), with similarity 0.5716091332094407
6: Dances with Wolves (1990), with similarity 0.5787399233167994
7: Firm, The (1993), with similarity 0.5857474094217832
8: Crimson Tide (1995), with similarity 0.590011387045958
9: Maverick (1994), with similarity 0.59125614918547
10: Congo (1995), with similarity 0.5921114226214383
11: Apollo 13 (1995), with similarity 0.595738152074041
12: Addams Family Values (1993), with similarity 0.5979860102939207
13: Client, The (1994), with similarity 0.598288921306952
14: True Lies (1994), with similarity 0.599321278588828


#User User Collaborative Filtering

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

user_item_pivot = popular_items.pivot(index='userId', columns='title', values='rating').fillna(0)
user_item_matrix = csr_matrix(user_item_pivot.values)

model_knn_user = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=15, n_jobs=-1)
model_knn_user.fit(user_item_matrix)

pick_random_user = np.random.choice(user_item_pivot.shape[0])

user_similarity, user_index = model_knn_user.kneighbors(user_item_pivot.iloc[pick_random_user, :].values.reshape(1, -1), n_neighbors=15)


##Making Reccomendations

In [ ]:
for i in range(1, len(user_similarity.flatten())):
    similar_user = user_item_pivot.index[user_index.flatten()[i]]
    similarity_score = user_similarity.flatten()[i]

    similar_user_movies = popular_items[popular_items['userId'] == similar_user]['title'].unique()

    recommended_movies = [movie for movie in similar_user_movies if user_item_pivot.loc[pick_random_user, movie] == 0]

    top_rated_movies = popular_items[popular_items['title'].isin(recommended_movies)].sort_values(by='rating', ascending=False).head(10)

    if not top_rated_movies.empty:
        print('{0}: User {1} (Similarity: {2}): Recommend top-rated movies - {3}'.format(
            i, similar_user, similarity_score, ', '.join(top_rated_movies['title'])))
    else:
        print('{0}: User {1} (Similarity: {2}): No new top-rated movie recommendations'.format(i, similar_user, similarity_score))

1: User 1119 (Similarity: 0.6493011058453938): Recommend top-rated movies - Young Frankenstein (1974), Clockwork Orange, A (1971), American Beauty (1999), L.A. Confidential (1997), Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), Clockwork Orange, A (1971), Star Wars: Episode VI - Return of the Jedi (1983), Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), Star Wars: Episode VI - Return of the Jedi (1983)
2: User 947 (Similarity: 0.6507460487635289): Recommend top-rated movies - Godfather, The (1972), Wallace & Gromit: The Wrong Trousers (1993), Memento (2000), Memento (2000), Memento (2000), Wallace & Gromit: The Wrong Trousers (1993), Memento (2000), Memento (2000), Memento (2000), Memento (2000)
3: User 85 (Similarity: 0.6549356444241962): Recommend top-rated movies - Princess Bride, The (1987), Dark Knight, The (2008), Lord of the Rings: The F